# <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2; text-align: center;">Topic Modeling</div>

An important task coming after data preprocessing and analysis (being the second step to virtually any NLP project) is topic modeling: a process that separates the existing data into multiple clusters, each of them representing a different topic. This is a crucial step in the process of understanding the data and extracting valuable insights from it. For this task, the team decided to use the BERTopic library, which is a topic modeling technique that leverages transformers model to create dense representations of the documents and then clusters them using HDBSCAN.

#### Used Embeddings

The embeddings used for topic modeling are taken from the project of digitalepidemiologylab in GitHub, which generated embeddings from COVID-19 tweets. These Embeddings are related to the BERT model, and a description about them can be found in the [official repository of digitalepidemoloylab's project](https://github.com/digitalepidemiologylab/covid-twitter-bert).

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Dependencies Imports</div>

In [2]:
# !pip install umap-learn
# !pip install hdbscan
# !pip install sentence-transformers
# !pip install bertopic
# !pip install nltk
# !pip install nbformat
# !pip install datamapplot
# !pip install dask[dataframe]


In [3]:
# When using CUDA, reduces VRAM usage
%env PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

env: PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True


In [4]:
import pandas as pd
import html
from umap import UMAP
from sklearn.decomposition import PCA
from hdbscan import HDBSCAN
from sentence_transformers import SentenceTransformer
from sklearn.feature_extraction.text import CountVectorizer
import datamapplot
import re

from bertopic import BERTopic
from bertopic.representation import KeyBERTInspired
from bertopic.vectorizers import ClassTfidfTransformer

2025-01-22 13:29:08.194684: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737548948.209010  745170 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737548948.213338  745170 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-22 13:29:08.228082: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Preprocessing</div>

Before starting to transform the data, some important preprocessing steps must be done in order to clean the data and maintain coherency in results with what has been done in the previous notebooks. The following steps were taken:

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. Data Loading</div>

First of all, the datasets inside the `data` folder must be loaded into the notebook. All train, test and validation datasets are fused into a single dataframe, which is stored inside the `tweets` variable.

In [5]:
DATA = 'data/'
test = pd.read_csv(DATA + 'Constraint_English_Test.csv', delimiter=';', encoding='utf-8')
train = pd.read_csv(DATA + 'Constraint_English_Train.csv', delimiter=';', encoding='utf-8')
val = pd.read_csv(DATA + 'Constraint_English_Val.csv', delimiter=';', encoding='utf-8')

tweets = pd.concat([train, val, test], ignore_index=True)
tweets.drop(columns=['id'], inplace=True)
tweets['tweet'] = tweets['tweet'].apply(html.unescape)


### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">2. Data Cleaning</div>

Of course, the imported data is not perfect and must undergo some cleaning. The following steps were taken:

1. First of all, the duplicate instances were removed from the dataset. For this purpose, the `drop_duplicates` method was used, which removes all rows that are exactly the same as another row.
2. The `text` column was cleaned from any presence of URLs, numbers, and indicators for a retweet (they are useless for topic modeling)
3. Some variables for storing real and fake tweets after processing were created.

In [6]:
processed_tweets = tweets.copy()
processed_tweets = processed_tweets.drop_duplicates(subset='tweet', keep='first')

# remove links
def filter(tweet:str):
    # https://stackoverflow.com/questions/3809401/what-is-a-good-regular-expression-to-match-a-url#3809435
    tweet = re.sub(r'https?:\/\/(www\.)?[-a-zA-Z0-9@:%._\+~#=]{1,256}\.[a-zA-Z0-9()]{1,6}\b([-a-zA-Z0-9()@:%_\+.~#?&//=]*)', '', tweet)
    tweet = re.sub(r'\d+', '', tweet) # remove numbers, not useful for topic modeling
    tweet = re.sub(r'RT @[a-zA-Z0-9_]{1,15}:', '', tweet)
    return tweet

processed_tweets['tweet_nolinks'] = processed_tweets['tweet'].apply(filter)

# Recalculate indexes
processed_tweets['tweet'].reset_index(drop=True, inplace=True)
real_tweets = processed_tweets[processed_tweets['label'] == 'real']['tweet_nolinks']
fake_tweets = processed_tweets[processed_tweets['label'] == 'fake']['tweet_nolinks']

processed_tweets

,tweet,label,tweet_nolinks
0,The CDC currently reports 99031 deaths. In gen...,real,The CDC currently reports deaths. In general ...
1,States reported 1121 deaths a small rise from ...,real,States reported deaths a small rise from last...
2,Politically Correct Woman (Almost) Uses Pandem...,fake,Politically Correct Woman (Almost) Uses Pandem...
3,#IndiaFightsCorona: We have 1524 #COVID testin...,real,#IndiaFightsCorona: We have #COVID testing la...
4,Populous states can generate large case counts...,real,Populous states can generate large case counts...
...,...,...,...
10695,#CoronaVirusUpdates: State-wise details of Tot...,real,#CoronaVirusUpdates: State-wise details of Tot...
10696,Tonight 12(midnight) onwards Disaster Manageme...,fake,Tonight (midnight) onwards Disaster Management...
10697,296 new cases of #COVID19Nigeria; Plateau-85 E...,real,new cases of #COVIDNigeria; Plateau- Enugu- O...
10698,RT @CDCemergency: #DYK? @CDCgov’s One-Stop Sho...,real,#DYK? @CDCgov’s One-Stop Shop for #COVID Reso...


Just for description purposes, the resulting dataset has a total of 3 features and 10699 instances.

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Data Transformation (Embeddings)</div>

In order to extract the different topics, a set of pre-trained models, called embeddings, can be used to transform the data into a format that can be used by the topic modeling algorithm. The adecuacy of the embeddings to the domain of the data is paramount for the well-performing of the topic modeling algorithm. In this case, the embeddigs used will be `covid-twitter-bert`, a model provided by digitalepidemiologylab.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. Load Embeddings</div>

In order to correctly load the embeddings, the library `sentence_transformers` (derivate from the `transformers` library) was used. The embeddings are loaded from the `covid-twitter-bert` model, which is a model trained on COVID-19 tweets. The embeddings are loaded into the `embeddings_real` and `embeddings_fake` variables, which store the embeddings for the real and fake tweets, respectively.

In [7]:
from sentence_transformers import SentenceTransformer
# Load the pre-trained model
sentence_model = SentenceTransformer('digitalepidemiologylab/covid-twitter-bert-v2')

# Encode the tweets
embeddings_real = sentence_model.encode(real_tweets.reset_index(drop=True).tolist(), show_progress_bar=True, batch_size=16)
embeddings_fake = sentence_model.encode(fake_tweets.reset_index(drop=True).tolist(), show_progress_bar=True, batch_size=16)

No sentence-transformers model found with name digitalepidemiologylab/covid-twitter-bert-v2. Creating a new one with mean pooling.
Batches: 100%|██████████| 319/319 [05:27<00:00,  1.03s/it]


### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">2. Dimensionality Reduction</div>

The embeddings have a high dimensionality, which can be a problem for the topic modeling algorithm. In order to reduce the dimensionality of the embeddings, the `UMAP` algorithm was used. The embeddings are reduced to 10 dimensions by merging dimensions using the cosine similarity metric. Since some errors appeared on the UMAP algorithm with fake tweets, a sepparate UMAP model was trained for fake tweets.

In [8]:
# Step 2 - Reduce dimensionality
umap_model = UMAP(n_neighbors=15, n_components=10, metric='cosine', random_state=None, low_memory=True)
umap_model_fake = UMAP(n_neighbors=min(len([tweet for tweet in processed_tweets[processed_tweets["label"] == "fake"]["tweet"]]) - 1, 15), n_components=10, metric='cosine', random_state=42, low_memory=True)

With this step, the Dimensionality reductor has been created and is ready to be used in the creation of the BERTopic model.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">3. Embedding Clustering</div>

Another important step of the BERTopic pipeline is the clustering of the embeddings. The embeddings are clustered using the `HDBSCAN` algorithm, which in this case uses the Euclidean distance metric to form the clusters. Again, once loaded, the clustering algorithm is ready to be used in the creation of the BERTopic model.

In [9]:
# Step 3 - Cluster reduced embeddings
cluster_model = HDBSCAN(min_cluster_size=15, metric='euclidean', cluster_selection_method='eom', prediction_data=True)

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">4. Topic Tokenization</div>

The last step of the BERTopic pipeline is the tokenization of the topics. The topics are tokenized using the `CountVectorizer` algorithm, which is a simple algorithm that counts the number of times each token appears in the topics. Once loaded, like with the rest of the models, the tokenization model is ready to be used in the creation of the BERTopic model.

In [10]:
# Step 4 - bag-of-words representation
vectorizer_model = CountVectorizer(stop_words="english", ngram_range=(1,2))

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">5. Topic Representation</div>

The last step of the BERTopic pipeline is the representation of the topics. The topics are represented using the `TfidfVectorizer` algorithm. Once loaded, like with the rest of the models, the representation model is ready to be used in the creation of the BERTopic model.

In [11]:
# Step 5 - Create topic representation (extract topic words)
ctfidf_model = ClassTfidfTransformer(reduce_frequent_words=True)

Additionally, since the names of the topics can be confusing at times, the team decided to use a pretrained LLM model to generate a summary of the topics from the documents and the keywords generated from the topics. The chosen model was `flan-t5-base` from Google's T5 models.

Using the `transformers` library (the base library for the `sentence_transformers` library), the model was loaded inside a pipeline with the task of generating text from some input text. Next, a prompt was created for the model to generate the summaries of the topics. Then, the whole pipeline is stored inside a representation model that can be integrated into the BERTopic model.

In [12]:
from transformers import pipeline
from bertopic.representation import TextGeneration
from bertopic.representation import MaximalMarginalRelevance

# Load a pre-trained model and tokenizer
model_name = 'google/flan-t5-base'  # You can replace this with other summarization models
# Create Maximal Marginal Relevance model
mmr = MaximalMarginalRelevance(diversity=0.3)

# Create Text Generator

generator = pipeline(
    model=model_name,
    task='text2text-generation',
)

prompt = """"
You are a helpful, respectful and honest assistant for labeling topics.
I have a topic that contains the following documents:
[DOCUMENTS]

The topic is described by the following keywords, separated by "_" characters: '[KEYWORDS]'.

Based on the information about the topic above, please create a short (three-word maximum) label of this topic.
Make sure you to only return the label and nothing more.

Everyone is counting on you to provide a good label for this topic.
"""
flant5 = TextGeneration(generator, prompt=prompt)
representation_model = {
    "Flan5Base": flant5,
    "MMR": mmr
}

Device set to use cpu


## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Creation and Training of BERTopic Models</div>

Once all the necessary models have been loaded and created, the BERTopic model can be created and trained. The BERTopic model is created using the `BERTopic` class from the `bertopic` library, which is a library that uses the BERT model to create dense representations of the documents and then clusters them using HDBSCAN. Of course, two BERTopic models are created, one for the real tweets and one for the fake tweets.

In [13]:
# All steps together
topic_model_real = BERTopic(
  embedding_model=sentence_model.to('cpu'),          # Step 1 - Extract embeddings
  umap_model=umap_model,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english',
  low_memory=True,
)

topic_model_fake = BERTopic(
  embedding_model=sentence_model.to('cpu'),          # Step 1 - Extract embeddings
  umap_model=umap_model_fake,                     # Step 2 - Reduce dimensionality
  hdbscan_model=cluster_model,              # Step 3 - Cluster reduced embeddings
  vectorizer_model=vectorizer_model,        # Step 4 - Tokenize topics
  ctfidf_model=ctfidf_model,                # Step 5 - Extract topic words
  representation_model=representation_model, # Step 6 - (Optional) Fine-tune topic representations
  calculate_probabilities = True,
  min_topic_size = 50,
  n_gram_range=(1, 2),
  verbose = True,
  language='english',
  low_memory=True,
)

In [14]:
# Clear the cache to avoid running out of VRAM (Tested on 4GB)
import torch
torch.cuda.empty_cache()
# cpu set as the default device because of the high VRAM use
torch.set_default_device('cpu')

Now that the models are created, it is time to train them on the real and fake tweets, respectively, before being able to extract relevant results from them.

In [15]:
# Trainning process
topics_real, probs_real = topic_model_real.fit_transform([tweet for tweet in real_tweets], embeddings_real)

2025-01-20 23:25:35,507 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
/home/miguelxd/.local/lib/python3.10/site-packages/numba/np/ufunc/parallel.py:371: NumbaWarning: The TBB threading layer requires TBB version 2021 update 6 or later i.e., TBB_INTERFACE_VERSION >= 12060. Found TBB_INTERFACE_VERSION = 12050. The TBB threading layer is disabled.
  warnings.warn(problem)
2025-01-20 23:25:58,490 - BERTopic - Dimensionality - Completed ✓
2025-01-20 23:25:58,491 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-20 23:25:59,327 - BERTopic - Cluster - Completed ✓
2025-01-20 23:25:59,330 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 71/71 [01:30<00:00,  1.27s/it]
2025-01-20 23:28:52,992 - BERTopic - Representation - Completed ✓


In [16]:
topics_fake, probs_fake = topic_model_fake.fit_transform([tweet for tweet in fake_tweets], embeddings_fake)

2025-01-20 23:28:53,231 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2025-01-20 23:29:00,017 - BERTopic - Dimensionality - Completed ✓
2025-01-20 23:29:00,018 - BERTopic - Cluster - Start clustering the reduced embeddings
2025-01-20 23:29:00,646 - BERTopic - Cluster - Completed ✓
2025-01-20 23:29:00,649 - BERTopic - Representation - Extracting topics from clusters using representation models.
100%|██████████| 55/55 [01:10<00:00,  1.28s/it]
2025-01-20 23:31:13,107 - BERTopic - Representation - Completed ✓


Once the models are trained, the topic labels are replaced by those generated by the LLM model, which are (a little) more descriptive and easier to understand.

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Results for Real Tweets</div>

It is time to start extracting the results from the BERTopic models. That is, extracting the formed topics and key information about them.



### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. DataFrame Visualization</div>

The first thing to do is to visualize the created topics inside a dataframe. This will be done for both the real and fake tweets.

In [17]:
topic_model_real.get_topic_info()

,Topic,Count,Name,Representation,Flan5Base,MMR,Representative_Docs
0,-1,1334,-1_rt_cdc_contact_fatality,"[rt, cdc, contact, fatality, learn, travel, ne...","[covid deaths, , , , , , , , , ]","[fatality, new york, indiafightscorona, secret...",[#IndiaFightsCorona: Five states account for a...
1,0,513,0_says_restrictions_uk_england,"[says, restrictions, uk, england, coronavirus,...","[covid restrictions, , , , , , , , , ]","[kayburley, boris johnson, government, lockdow...",[Sir Keir Starmer asks the prime minister abou...
2,1,494,1_day average_numbers_hospitalizations_states ...,"[day average, numbers, hospitalizations, state...","[COVID- cases reported in _, , , , , , , , , ]","[day average, states reported, cases today, po...",[The upward trend holds for death data. Though...
3,2,253,2_drtedros_countries_drtedros covid_covax,"[drtedros, countries, drtedros covid, covax, v...","[covid vaccines, , , , , , , , , ]","[drtedros covid, covax, covax facility, act ac...",[The Access to #COVID Tools (ACT) Accelerator ...
4,3,149,3_face covering_covering_hands_cloth,"[face covering, covering, hands, cloth, wear, ...","[#COVID, , , , , , , , , ]","[face covering, cloth face, wash hands, spread...",[You can help slow the spread of #COVID. Pract...
...,...,...,...,...,...,...,...
66,65,17,65_stress_cope stress_ways cope_stress covid,"[stress, cope stress, ways cope, stress covid,...","[#COVID, , , , , , , , , ]","[ways cope, stress covid, support health, mino...",[It is common to feel more stress during the #...
67,66,16,66_pooled_requires complex_reassurance_testing...,"[pooled, requires complex, reassurance, testin...","[#COVID, , , , , , , , , ]","[testing requires, complex staff, provide reas...",[(/) It is heartening to see UK introduce the ...
68,67,16,67_forecasts_forecasts suggest_forecast_suggest,"[forecasts, forecasts suggest, forecast, sugge...","[#COVID deaths in the US, , , , , , , , , ]","[forecasts suggest, june forecasts, likely rep...",[As of June forecasts suggest states are lik...
69,68,16,68_week projecting_arrivals departures_project...,"[week projecting, arrivals departures, project...","[_, , , , , , , , , ]","[week projecting, arrivals departures, capacit...",[There are currently people in managed isolat...


Overall, a total of around 50 topics for real tweets has been created. The LLM did a deplorable job in providing summary labels for some of the topics, lots of them are not very clear due to the repetitiveness of the summary labels. Since Most of the topics are labeled as "COVID" alone or with a few other words, the names of the labels will be discarded and generic topic names will be chosen instead. For the labels that are clear, we can see that many of them are focused on informative purposes, like restriction updates and information about arising cases.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">2. Distance Map</div>

The next chosen representation is a distance map, which is a representation of the topics in a 2D space. This representation is useful for understanding the relationships between the topics and how they are distributed in the semantic space.

In [18]:
# Topics visualization
topic_model_real.visualize_topics()

It is visible that the vast majority of the topic distribution is concentrated in greatly separated clusters, which have a huge topic concentration each, with a kind of 'nexus' in the middle of the map. This gives out the idea that there are well-separated topics, with a few topics that are the most common among the tweets and have a strong relationship with each other.

While ideal to see the distribution and semantic variety of the topics, it is not very useful for understanding the topics themselves. For that, a different representation must be used.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">3. Hierarchical Visualization</div>

In order to better visualize how topics relate to each other, the hierarchical visualization is ideal for this task and has hence been chosen by the team. This visualization shows the relationships between the topics in a hierarchical manner in a tree-like structure.

In [19]:
# hierarchical topic visualization
topic_model_real.visualize_hierarchy() #false for default labels

We can see a complex hierarchy of topics. Some of the most interesting insights this visualization leaves us is that there are specific and closely related topics for tests, cases, hashtags, and worldwide restriction updates.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">4. Document Cluster Visualization</div>

This kind of visualization is needed in order to understand how documents relate each other and find out possible patterns in their themes. In order to provide this representation, a reduced version of the embeddings shall be provided, which is done by the UMAP algorithm.

In [20]:
# visualize documents
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings_real)

# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
topic_model_real.visualize_documents([tweet for tweet in real_tweets], reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=True)

Just as with the Distance Map, the Document Cluster Visualization shows a nexus in the center of the representation, where most of the topics meet, whilst the rest of the topics are scattered far apart from the center. This gives out the idea that there are a few main topics that are the most discussed, and that the rest of the topics are not as relevant.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">4. Heat Map Visualization</div>

The Heat Map Representation creates a similarity matrix among the topics. This representation is useful for understanding clearly the similarities within the topics.

In [21]:
topic_model_real.visualize_heatmap(n_clusters=20, width=1000, height=1000)

The heat map offers the visualization of a set of topics that have multiple similarity levels with others. Something interesting is that topics related to COVID cases and COVID in India (just in the lower rows of the dataset) do not have a great semantic relation to any of the other topics (they are the most close to white in the whole heatmap).

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">5. Topic Word Score Bar Chart</div>

This representation shows the most imporant and relevant words for each topic. This is useful for understanding the main themes of each topic.

In [31]:
topic_model_real.visualize_barchart(top_n_topics=8)

For the real tweets, the most important or relevant topics show words that correspond to restrictions, information, and case reports. Most of these words belong to very neutral topics, which do not try to convey any kind of sentiment and are focused on the conveyed information about the main point of interest: the virus.

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Results for Fake Tweets</div>

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">1. DataFrame Visualization</div>

In [23]:
topic_model_fake.get_topic_info()

,Topic,Count,Name,Representation,Flan5Base,MMR,Representative_Docs
0,-1,1984,-1_died_virus_hospital_man,"[died, virus, hospital, man, says, cdc, deaths...",[COVID: kin of those who died of COVID- can cl...,"[virus, hospital, man, says, people, home, loc...",[USA just discovered the man who manufactured ...
1,0,323,0_chloroquine_hydroxychloroquine_vitamin_zinc,"[chloroquine, hydroxychloroquine, vitamin, zin...","[_, , , , , , , , , ]","[chloroquine, hydroxychloroquine, drug, quinin...",[Mask wearers beware. A caller to a radio talk...
2,1,177,1_news_news government_boris_lockdown,"[news, news government, boris, lockdown, news ...","[kylie minogue, , , , , , , , , ]","[news, news coronavirus, news boris, boris joh...",[NEWS! ‘Loss of taste’ added to COVID- symptom...
3,2,159,2_gates_patent_vaccine_gates foundation,"[gates, patent, vaccine, gates foundation, dna...","[vaccines, , , , , , , , , ]","[microchip, covid vaccine, vaccination, patent...",[Multiple Facebook posts shared hundreds of ti...
4,3,147,3_pelosi_north carolina_vote_says,"[pelosi, north carolina, vote, says, hoax, emp...","[nancy pelosi, , , , , , , , , ]","[north carolina, lewis funeral, pelosi daughte...",[At John Lewis funeral Obama told a room full ...
5,4,143,4_factcheck_coronacheck_misinformation_datosco...,"[factcheck, coronacheck, misinformation, datos...","[covid misinformation, , , , , , , , , ]","[factcheck, datoscoronavirus, factchecknet, co...",[As India detected more #COVID cases in March-...
6,5,106,5_donaldtrump_donaldtrump coronavirus_coronavi...,"[donaldtrump, donaldtrump coronavirus, coronav...","[trump coronavirus, , , , , , , , , ]","[donaldtrump coronavirus, donaldtrump china, t...","[President Trump Says He Will Never, Ever Get ..."
7,6,100,6_water_hot_lemon_hot water,"[water, hot, lemon, hot water, drinking, salt,...","[_ ., , , , , , , , , ]","[hot water, drinking water, kill virus, stomac...",[Corona syrup “Prevention is better than cure”...
8,7,99,7_nobel_honjo_tasuku_tasuku honjo,"[nobel, honjo, tasuku, tasuku honjo, lab, laur...","[COVID- engineered at wuhan lab, , , , , , , ,...","[tasuku, tasuku honjo, nobel laureate, japanes...",[TRUTH can Never be Kept Under The Carpet :\n#...
9,8,85,8_global pandemic_donald trump_donald_price,"[global pandemic, donald trump, donald, price,...","[obamacare, , , , , , , , , ]","[global pandemic, donald trump, paying price, ...",[#covid #coronavirus #coronaoutbreak Let me be...


Just like with the real tweets, the fake tweets can be divided into around 50 topics. Surprisingly, the frequency of use of the word "COVID" for labeling the topics is much lower than in the real tweets. This can be a clear indicative of the way disinformers elaborate their messages: They try to avoid the use of technical terms and focus on the emotional impact of the message.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">2. Distance Map</div>

In [24]:
topic_model_fake.visualize_topics()

One visible particularity on the distance map on fake tweets is that the centered nexus is much less concentrated of topics than in the real tweets. This can be a sign that disinformers may write about a wider range of topics than the real informers, which may be a sign of the way they try to reach a wider audience. The concentration and size of the scattered clusters is decent, whilst on the real tweets the clusters that deviate from the center of the map are much smaller and less concentrated.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">3. Hierarchical Clustering</div>

In [25]:
topic_model_fake.visualize_hierarchy()

The hierarchical view of topics shows very distinct semantic clusters, each with topics with labels that clearly stand out the relation between them. Among the topics, we can see topics that focus on the vaccine, animals, countries and even some particular individuals. These topics are very closely related, so it is not a coincidence that fake tweets try to point to all these other factors. These topics might not be relevant about giving out useful information about the virus, but it is rather related to giving out clues about what to blame for the pandemic.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">4. Document Cluster Visualization</div>

In [26]:
# visualize documents
reduced_embeddings = UMAP(n_neighbors=10, n_components=2, min_dist=0.0, metric='cosine').fit_transform(embeddings_fake)

# Visualize the documents in 2-dimensional space and show the titles on hover instead of the abstracts
# NOTE: You can hide the hover with `hide_document_hover=True` which is especially helpful if you have a large dataset
topic_model_fake.visualize_documents([tweet for tweet in fake_tweets], reduced_embeddings=reduced_embeddings, hide_annotations=True, hide_document_hover=True)

A remarkable finding when looking at this topic representation view is that there seems to be a topic whose documents scatter accross the whole cluster map, maybe even coinciding with some clusters of different colours. What this might mean is that there is something mentioned in each of the topics that is common to all of them.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">5. Heat Map Visualization</div>

In [27]:
topic_model_fake.visualize_heatmap(n_clusters=20, width=1000, height=1000)

Much of a curious fact, the heat map representation indicates that unlike on real tweets, the overall level of interrelation between topics is much lower. The heatmap blue tones are much more present in the real tweets' heatmap. This could be a sign that disinformers tend to write about a wider and more disconnected topics from the main topic.

### <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4a44c2;">6. Topic Word Scores Bar Chart</div>

In [30]:
topic_model_fake.visualize_barchart(top_n_topics=6)

A very small proportion of words related to the virus itself are observable in comparison to the real tweets. Once again, the focus of the most relevant topics is on particular individuals, countries and even some other random data not related to the virus (there is an entire topic just focused on Donald Trump). This significant difference is a clear indicative that disinformers have a different way of writing about the core topic, they have other interests than really getting to the core information.

## <div style="border: 3px solid #FFFFF; padding: 10px; border-radius: 5px; background-color: #4484c2;">Conclusions</div>

Topic modeling has clearly shown there are significant differences between the way real and fake tweets are written. The real tweets are focused on the core information about the virus, like restrictions, cases, and information about the virus itself. On the other hand, fake tweets are focused on a wider range of topics, like the vaccine, animals, countries and, in several cases, particular individuals and high world authorities. These differences could be critical to determine in the future if a message is real or fake, and could be used to develop a model that can classify tweets as real or fake based on the topics they talk about.

On the other side, this is just a semantic analysis on the tweets and itself alone may not be sufficient to provide a real conclusion . Some more analysis could lead to additional evidence to support it.